<a href="https://colab.research.google.com/github/passiona2z/kakaotalk_wordcloud/blob/master/colab_kakao_word_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 워드 클라우드 (국문)

#### 필요한 패키지 불러오기

In [ ]:
# 한글 분석 라이브러리 설치.
!pip install konlpy

In [ ]:
# 한글 (나눔) 글꼴 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplitlib -rf

In [103]:
from wordcloud import WordCloud
import konlpy
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')     # 한글 깨짐 처리

from PIL import Image                         # Pillow 패키지의 영상 핸들링 클래스


import os                                     # 파일 확인
import natsort                                # 파일 정렬

#### 데이터 읽어오기

In [ ]:
# 파일명 가져오기

PATH = '/content/'                   # 코랩 파일에 바로 업로드하면 경로변경 불필요
file_list = os.listdir(PATH)
text_list = list()

for file in file_list:
    if file.split(".")[-1] == 'txt':  # check
        text_list.append(file)

text_list = natsort.natsorted(text_list) # natsort 정렬

text_list

In [105]:
# 카카오톡 데이터 불러오기

text_list_all = []

for i in text_list :
   with open('/content/'+i, 'r', encoding='utf-8-sig') as f :
      my_text = f.readlines()
      text_list_all.extend(my_text)  # extend    

In [ ]:
text_list_all[:10]                   # 데이터 확인

#### 전처리:

In [ ]:
# 텍스트 부분 발췌

my_line = [ a_line.split(':') for a_line in text_list_all]

my_line_word = []

for a_line in my_line :

    try :
        a_line = a_line[2] 

    except : 
        continue            # try - except

    my_line_word.append(a_line)     

my_line_word[:10]           # 텍스트 확인

In [108]:
# 불용어 정의하기 (추가가능)
no_meaning = 'ㅠ|ㅜ|ㅡ|ㅋ|ㅎ|'
no_meaning += '이모티콘|사진|이양|보오|하나|아앙|가구|거양|나이'

In [109]:
# 전처리
my_line_clean = []

for a_line in my_line_word :

    a_line = re.sub(no_meaning, ' ', a_line)     # 특별한 의미 없는 단어 스페이스로 대체.
    a_line = re.sub('\W+',' ', a_line)           # 특수 문자 스페이스로 대체.
    a_line = re.sub('[-!?()>~.,]',' ',a_line)    # 특수문자 스페이스로 대체.
    a_line = re.sub('\d+',' ', a_line)           # 숫자 스페이스로 대체.
 
    a_line = re.sub('\n',' ',a_line)             # line return 스페이스로 대체.
    a_line = re.sub('[\[\]]', ' ',a_line)        # 대괄호 스페이스로 대체.
    a_line = re.sub('[a-zA-Z]',' ',a_line)       # 영문 스페이스로 대체.
    a_line = re.sub('\s+', ' ', a_line)          # 잉여 스페이즈 줄임.

    my_line_clean.append(a_line)


In [ ]:
# 몇개만 출력해 본다.
my_line_clean[:10]

#### 한글 단어 추출(라이브러리)

In [111]:
my_tagger = konlpy.tag.Okt()   # 라이브러리 객체

In [112]:
# 명사 추출.
my_words = []
for a_line in my_line_clean:
    my_words.extend(my_tagger.nouns(a_line))  # 명사 # extend
    # my_words.extend(my_tagger.morphs(a_line)) # 모든 형태소 # (부가옵션) stem=True 

In [113]:
# 단음절 제거 (주관적 판단)
my_words_2 = []
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2 += [a_word]

# List comprehension 방법 사용.
# my_words_2 = [a_word  for a_word in my_words if len(a_word) > 1]

#### 키워드 추출

In [ ]:
# Series 로 변환.
my_series = pd.Series(my_words_2)
my_series

In [ ]:
# 도수 분포표 Top 50
my_word_counts = my_series.value_counts().sort_values(ascending=False)
my_word_counts[:50]  

In [117]:
# 딕셔너리로 변환
my_dict = {}
for an_index, a_value in zip(my_word_counts.index,my_word_counts.values):
    my_dict[an_index] = a_value

#### 워드 클라우드 기본형 생성

In [ ]:
# 워드클라우드의 요구사항.
# a_long_sentence = ' '.join(my_words_2)

In [118]:
wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic',background_color='white', max_words=50)              # 바탕색, 단어 개수 등 설정
# wc.generate(a_long_sentence)
wc.generate_from_frequencies(my_dict)
# wc.words_

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(wc)
plt.axis("off")                                    # 축을 꺼줌.
plt.show()

#### 워드 클라우드 마스크(하트) 사용형 생성

In [120]:
# 백그라운드 마스크
img = Image.open('/content/background_3.png')                    # 하트.
back_mask = np.array(img)                                        # 넘파이 배열로 변환

In [ ]:
wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic',background_color='white', max_words=50, mask=back_mask)          
wc.generate_from_frequencies(my_dict)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(wc) 
plt.axis("off")                                    # 축을 꺼줌.
plt.show()